# 使用Ollama和LangChain搭建本地大语言模型聊天机器人
动手时间到！
本笔记将演示如何使用Ollama和LangChain在你的电脑上完全本地化地运行一个大语言模型（LLM）。
- **Ollama** 是一个可以让我们在本地机器上运行LLM的工具。
- **LangChain** 是一个流行的软件开发工具包（SDK），它封装了可复用的代码，使与LLM的交互变得更加简单。
## 1. 准备工作
在运行本笔记之前，你需要先安装Ollama并下载一个模型。请按照以下步骤操作。
### 1.1 安装 Ollama
**macOS:**
- 从 [ollama.ai](https://ollama.ai) 下载安装程序。
- 打开下载的文件，并按照安装提示进行操作。
**Windows:**
- 从 [ollama.ai](https://ollama.ai) 下载安装程序。
- 运行安装程序，并按照说明进行操作。
**Linux:**

In [ ]:
%%bash
curl -fsSL https://ollama.ai/install.sh | sh

验证你的安装（适用于Windows, Mac, 和 Linux）:

In [ ]:
%%bash
ollama --version

### 1.2 下载 TinyLlama 模型
现在，我们来下载 **TinyLlama** 模型（约600MB）。Ollama的模型库里有包括DeepSeek在内的众多模型可供选择，但TinyLlama是体积最小的模型之一，非常适合用于演示目的：

In [ ]:
%%bash
ollama pull tinyllama:1.1b

## 2. 配置Python环境
接下来，安装所需的Python包。我们将安装 **LangChain**，这是与LLM交互最流行的库之一。而 **langchain-ollama** 这个包则专门用于连接我们本地的Ollama模型。

In [ ]:
%%bash
pip install langchain langchain-ollama

## 3. 使用LangChain与LLM交互
下面的代码将演示如何与我们本地托管的 `tinyllama` 模型进行交互。我们将向模型发送一条简单的消息：`“我热爱编程”`。
请仔细阅读代码中的注释，以便更好地理解每一步的作用。

In [ ]:
# 导入所需库
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage
# 初始化LLM。在这里我们选择要使用的模型和“温度”（temperature），温度控制输出的随机性。
# llm 对象将是我们调用模型的接口。
llm = ChatOllama(
    model="tinyllama:1.1b",
    temperature=0,  # 温度设为0，表示我们希望得到更确定、更稳定的输出
)
# 我们需要向模型提供一个消息列表。第一个消息通常是“系统”消息，用于设定模型的角色和行为。
# 第二个消息是“人类”消息，即我们想对模型说的话。
messages = [
    (
        "system",
        "你是一个乐于助人的助手。",
    ),
    ("human", "我热爱编程。"),
]
# 将消息列表发送给模型并获取响应。响应是一个AIMessage对象，其中包含了模型回复的内容。
ai_msg = llm.invoke(messages)
# 显示模型的回复内容
print(ai_msg.content)

## 4. 测试不同的提示（Prompts）
让我们尝试几个不同的例子，通过改变消息数组和内容，看看模型能做些什么：

In [ ]:
# 请求解释一个概念
messages = [
    (
        "system",
        "你是一个乐于助人且知识渊博的AI助手。",
    ),
    ("human", "用简单的语言解释一下什么是神经网络。"),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
# 请求编写代码
messages = [
    (
        "system",
        "你是一位资深的Python程序员。",
    ),
    ("human", "写一个函数来检查一个字符串是否是回文。"),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

## 5. 创建一个基础的聊天机器人（无记忆）
现在，我们来创建一个简单的聊天界面，以便与模型进行互动。在Jupyter Notebook中，我们可以用不同的代码单元格来模拟每一次的交互。跟着下面的步骤操作，你会更容易理解。

In [ ]:
# 设置：首先运行此单元格
from langchain_ollama import ChatOllama
# 初始化LLM
llm = ChatOllama(
    model="tinyllama:1.1b",
    temperature=0
)
# 定义系统提示
system_prompt = "你是一个乐于助人的助手。"
print("-" * 30)

现在，我们创建第一个交互单元格，告诉模型我们最喜欢的颜色。请注意，由于我们使用的是一个非常小的模型，它的回答可能有些奇怪，但它应该能表明它理解了你最喜欢的颜色是蓝色。

In [ ]:
# 第一次交互
user_message = "我最喜欢的颜色是蓝色。"
# 为本次交互创建一个全新的消息列表
messages = [
    ("system", system_prompt),
    ("human", user_message)
]
# 获取响应
ai_msg = llm.invoke(messages)
print(f"你: {user_message}")
print(f"助手: {ai_msg.content}")

现在，我们来问一个后续问题：

In [ ]:
# 第二次交互
user_message = "我最喜欢的颜色是什么？"
# 同样，为本次交互创建一个全新的消息列表
messages = [
    ("system", system_prompt),
    ("human", user_message)
]
# 获取响应
ai_msg = llm.invoke(messages)
print(f"你: {user_message}")
print(f"助手: {ai_msg.content}")

你会发现，模型**不记得**我们最喜欢的颜色！为什么会这样呢？像ChatGPT这样的工具明明可以记住整个对话的上下文。
这就是**记忆（Memory）** 的概念。LLM本身是**无状态的**，它不会自动记住过去的对话。我们需要在应用层面进行设计，才能让它拥有记忆。
下面，我们来探讨如何实现这一点，以及像ChatGPT这样的工具是如何管理对话记忆的。
## 6. 理解对话记忆的工作原理
实现记忆最简单的方法，就是在每次调用模型时，**将完整的对话历史记录传递给它**，而不仅仅是当前这一条消息：

In [ ]:
from langchain_ollama import ChatOllama
from langchain.schema import SystemMessage, HumanMessage, AIMessage
# 像之前一样初始化LLM
llm = ChatOllama(
    model="tinyllama:1.1b",
    temperature=0
)
# 定义一个列表来存储对话历史（总是以系统提示开始）
# conversation 是一个消息对象列表。SystemMessage 是代表系统消息的类。
conversation = [SystemMessage(content="你是一个乐于助人的助手。")]
# 定义一个名为 chat 的函数，它接收用户输入作为参数，用于与模型交互。
def chat(user_input: str):
    # 1) 将用户的消息添加到对话历史中
    # 现在 conversation 列表中包含了系统消息和用户的消息。
    conversation.append(HumanMessage(content=user_input))
    
    # 2) 用完整的对话历史（系统消息 + 用户消息）调用模型
    ai_msg = llm.invoke(conversation) # LangChain v0.2+ `llm.invoke` is preferred
    
    # 3) 显示本次交互的内容
    print(f"你: {user_input}")
    print(f"助手: {ai_msg.content}")
    print(f"[对话长度: {len(conversation) + 1} 条消息]" ) # +1 是因为包含了AI的响应
    
    # 4) 将AI的响应也添加到历史记录中
    # 现在 conversation 列表包含了系统、用户和AI三方的消息，为下一次交互做好了准备。
    conversation.append(ai_msg)
    
    return ai_msg.content
print("有记忆的聊天")
print("-" * 30)

让我们再告诉它一次我们最喜欢的颜色：

In [ ]:
# 第一次交互 - 告诉它最喜欢的颜色
chat("我最喜欢的颜色是蓝色。")

现在，看看它是否记住了：

In [ ]:
# 第二次交互 - 测试记忆
chat("我最喜欢的颜色是什么？")

这次，模型应该能正确地记住你的颜色了！这是因为我们手动维护了完整的对话历史记录。
## 7. 上下文窗口的挑战
虽然我们简单的记忆方案在初期有效，但它有一个致命的局限：**上下文窗口（Context Window）**。
### 理解上下文窗口
每个语言模型都有一个固定的“上下文窗口”——即它一次能处理的最大令牌（tokens，大致可以理解为单词或词组）数量：
- **TinyLlama**: 约 2,048 令牌 (大约 1,500 词)
- **GPT-3.5**: 约 4,096 令牌
- **GPT-4**: 根据版本不同，约 8,192 到 128,000 令牌
- **Claude 3**: 高达 200,000 令牌
随着对话的进行，我们的对话历史会越来越长，最终会超出这个限制。当这种情况发生时：
1. 模型将无法“看到”超出窗口大小的早期消息。
2. 它实际上“忘记”了对话的开头部分。
3. 处理更长的上下文会导致响应变慢。
4. 如果超出上下文窗口，你可能会收到API错误。
## 8. 上下文窗口问题的解决方案
在像ChatGPT这样的生产级系统中，有几种技术可以解决上下文窗口的限制：
### 1. 滑动窗口 (Sliding Context Window)
这种方法只保留最近的 N 条消息，外加最初的系统提示。
**例如：** 如果你限制保留10条消息，而对话已经进行了15条，系统就会丢弃最老的5条消息（系统提示除外）。
这就像一场你只能记住最近几分钟谈话内容的对话。它实现简单，但会丢失所有较早的信息。
### 2. 对话摘要 (Summarization)
这种技术将对话的早期部分压缩成摘要，以节省令牌空间。
**例如：** 经过10轮对话后，系统可能会用一条摘要来替换这20条消息：`“用户自称Alex，询问了关于神经网络的问题。助手解释了神经网络，并提供了一个检查回文的Python代码示例。”`
这在保留关键信息的同时，显著减少了令牌的使用。
### 3. 基于数据库的会话管理 (Database-Backed Session Management)
这种方法使用数据库来存储与唯一用户会话相关联的完整对话历史。其工作原理如下：
1. **创建会话**: 当用户开始聊天时，系统会创建一个唯一的会话ID（如 `session_abc123`）。
2. **存储消息**: 用户和AI的每一条消息都被存储在与该会话ID关联的数据库表中。
3. **管理上下文窗口**: 在准备向LLM发送提示时，系统会：
   - 从数据库中检索该会话的所有消息。
   - 应用策略（如滑动窗口或摘要）来使内容适应上下文窗口。
   - 将优化后的对话发送给模型。
   - 将新的响应存回数据库。
**实践中的例子：**
- 一个用户与ChatGPT聊了几个小时。
- 他的会话ID `user_789` 在数据库中现在有200条消息。
- 当他发送第201条消息时，系统会：
  - 从数据库中检索出全部200条历史消息。
  - 选择最重要的部分以适应上下文窗口。
  - 从模型获取响应。
  - 将第201条消息和模型的响应添加到数据库中。
  - 即使模型在本次交互中只“看到”了最近的一部分对话，完整的历史记录仍然保留在数据库中。
这就是像ChatGPT这样的服务能够实现跨越长时间、甚至关闭浏览器后重新打开也能“记住”对话内容的原因。
## ChatGPT通常使用什么策略？
ChatGPT采用了一种结合了多种技术的混合方法：
1. **主要技术:** 它利用一个非常大的上下文窗口（在GPT-4o中高达128K令牌）和基于数据库的会话管理，使其能够“记住”非常长的对话。
2. **对于超长对话:** 当对话甚至超出了这些慷慨的限制时，它会采用滑动窗口技术，并优先保留：
   - 系统提示/指令。
   - 最近的消息。
   - 信息密度高的消息。
   - 用户明确引用的消息。
3. **动态压缩:** 在某些版本中，ChatGPT还使用动态压缩算法，选择性地总结或移除对话中与当前交流似乎不太相关的部分。
尽管这些解决方案很有帮助，但任何模型在单次对话中能“记住”的内容仍然存在绝对的限制。这就是为什么即使是ChatGPT，在非常长的对话中有时也会“忘记”很早之前提到的事情。
## 9. 本地LLM的优势
使用像Ollama这样的工具在本地运行LLM有几个显著的优点：
1. **隐私性**: 你的所有数据都保留在自己的电脑上，永远不会离开。
2. **无API费用**: 你可以随心所欲地运行模型，无需为每次查询付费。
3. **离线使用**: 一旦模型下载完成，就不再需要互联网连接。
4. **无速率限制**: 只要你的硬件能承受，你可以运行任意数量的查询。
## 10. 进一步学习的资源
- [Ollama 文档](https://github.com/ollama/ollama/blob/main/README.md)
- [LangChain 文档](https://python.langchain.com/docs/get_started/introduction)
- [TinyLlama 模型信息](https://github.com/jzhang38/TinyLlama)